# Example

In [1]:
%load_ext autoreload
%autoreload 2

%env GS_BACKEND=native_add3
%env CUDA_VISIBLE_DEVICES=0

from omegaconf import OmegaConf

env: GS_BACKEND=native_add3
env: CUDA_VISIBLE_DEVICES=0


### [0] Load images

In [2]:
WS = "./demo/duck"
PRE_CFG_FN = "./profile/demo/demo_prep.yaml"
FIT_CFG_FN = "./profile/demo/demo_fit.yaml"

In [3]:
from mosca_precompute import load_imgs_from_dir

img_list, img_fns = load_imgs_from_dir(WS)
pre_cfg = OmegaConf.load(PRE_CFG_FN)
# pre_cfg.tap_chunk_size = 4096 # reduce this for smaller gpu

GS_BACKEND: native_add3
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### [1] Precompute Priors

In [4]:
from mosca_precompute import get_moca_processor, preprocess

moca_processor = get_moca_processor(pre_cfg)

loading raft ...
loading depthcrafter ...


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

`torch_dtype` is deprecated! Use `dtype` instead!
Using cache found in /root/.cache/torch/hub/yvanyin_metric3d_main
/opt/conda/envs/mosca/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


loading spatracker...


In [ ]:
preprocess(
    img_list=img_list,
    img_fns=img_fns,
    ws=WS,
    moca_processor=moca_processor,
    pre_cfg=pre_cfg,
)

### [2] Recon Stage


In [ ]:
from mosca_reconstruct import setup_recon_ws

fit_cfg = OmegaConf.load(FIT_CFG_FN)

logdir = setup_recon_ws(WS, fit_cfg=fit_cfg)
print(logdir)

In [ ]:
from lite_moca_reconstruct import static_reconstruct

static_reconstruct(WS, logdir, fit_cfg)

In [ ]:
from mosca_reconstruct import photometric_warmup

photometric_warmup(WS, logdir, fit_cfg) # this is optional, will auto skip by default.

In [ ]:
from mosca_reconstruct import scaffold_reconstruct

scaffold_reconstruct(WS, logdir, fit_cfg)

In [ ]:
from mosca_reconstruct import photometric_reconstruct

photometric_reconstruct(WS, logdir, fit_cfg)

### [3] viz and fps

In [ ]:
from mosca_evaluate import test_fps

test_fps(logdir)

In [ ]:
from mosca_viz import viz_main
import os.path as osp

viz_main(osp.join(logdir, "viz"), logdir, FIT_CFG_FN, N=5, back_ratio_3d=1.5)